In [1]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json
from openai import OpenAI

In [ ]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        client = OpenAI(
            base_url = "https://api.fanar.qa/v1",
            api_key  = os.environ["FANAR_API_KEY"],
        )
        client.default_params = {"model": "Fanar-C-1-8.7B"}    
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="azure"
deployment = APIKeyManager(model_type, "./azure.env")


In [ ]:
class PodcastOutlineGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model

    def generate_outline(self, topic, information,host_persona, guest_persona,):
        prompt = f"""
You are tasked with creating a clear and engaging structured outline for a podcast episode on the topic: '{topic}'.

Use the following personas clearly:
- Host: '{host_persona}'
- Guest: '{guest_persona}'

Base your outline on this provided information:
{information}

Organize your outline strictly in the following JSON format, using these exact keys:

{{
    "Intro1": {{
        "description": "Brief overview of the podcast and general welcome from the host.",
        "script": ["Engaging and enthusiastic introduction by the host."]
    }},
    "Intro2": {{
        "description": "Introduction of today's topic and brief bio of the guest speaker.",
        "script": ["Introduction to topic by host.", "Warm welcome and brief guest introduction."]
    }},
    "Points": {{
        "talking_points": {{
            "Point Title 1": {{
                "discussion": "Brief discussion of talking point.",
                "questions": [
                    "Relevant question 1?",
                    "Relevant question 2?"
                ],
                "response_hint": "Brief guidance on the desired guest response."
            }},
            "... Additional points as appropriate ..."
        }}
    }},
    "Con": {{
        "description": "Summary and reflection, leaving listeners with a thoughtful question.",
        "script": ["Powerful summary statement.", "Thought-provoking closing question for the audience."]
    }}
}}

Return only valid JSON adhering strictly to this structure without additional explanatory text.
Just return the output in json format without any seperate markers
"""
        response = deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a skilled podcast content planner."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content


In [ ]:
outline_generator = PodcastOutlineGenerator (deployment,"fanar")


In [5]:
# import PodcastOutlineGenerator
topic = "AI in Education"
information = (
    "AI is transforming education through personalized learning, automated grading, intelligent tutoring systems, "
    "and providing accessibility for students with disabilities. Challenges include ethical concerns, data privacy, "
    "and potential biases in algorithms."
)
host_persona = "Alex, an experienced educator"
guest_persona = "Jamie, a technology enthusiast and CEO of a startup"

outline_text = outline_generator.generate_outline(topic, information, host_persona, guest_persona)
print("Generated Outline:\n", outline_text)
# podcast_creator = PodcastCreator(topic, information, host_persona, guest_persona)




Generated Outline:
 {
    "Intro1": {
        "description": "Brief overview of the podcast and general welcome from the host.",
        "script": ["Welcome to the podcast! I'm Alex, and I’m thrilled to bring you engaging discussions on relevant topics in education."]
    },
    "Intro2": {
        "description": "Introduction of today's topic and brief bio of the guest speaker.",
        "script": ["Today, we’re discussing a groundbreaking topic: AI in Education—how technology is reshaping classrooms and paving the way for new possibilities.", "Joining us is Jamie, a technology enthusiast and the CEO of an innovative startup specializing in AI solutions for learning."]
    },
    "Points": {
        "talking_points": {
            "Point Title 1": {
                "discussion": "How AI is personalizing learning experiences for students.",
                "questions": [
                    "How does AI adapt to different student learning styles?",
                    "Could you give e

In [6]:

class DialogueEnhancer:
    def __init__(self, deployment,model="gpt-4o"):
        self.model = model

    def enhance_section(self, section_name, content, criteria, style_example=None):
        if style_example:
            prompt = (
                f"""Here is an example of the dialogue style you should follow:\n{style_example}\n\n"
                Given the following podcast section, modify it based on the following criteria: {criteria}

                Section: {section_name}
                Content: {content}

                Provide a compelling and engaging script."""
            )
        else:
            prompt = (
                f"""Given the following podcast section, modify it based on the following criteria: {criteria}

                Section: {section_name}
                Content: {content}

                Provide a compelling and engaging script."""
            )
        response = deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a professional podcast scriptwriter."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content


In [7]:
class SpontaneousDialogueGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
    def generate_dialogue(self, section, section_content, host_persona, guest_persona, style_example=None):
        if style_example:
            prompt = (
                f"""Here is an example of the dialogue style you should follow:\n{style_example}\n\n"
                f"Now, transform the following podcast section into a spontaneous dialogue between host {host_persona} and guest {guest_persona}, naturally including conversational fillers, pauses with possible duration in brackets [pause: 2s], and interaction tags like <laughter>, <overlap>, <interruption>, <hesitations>.\n\n"
                f"Section: {section}\nContent: {section_content}"""
            )
        else:
            prompt = (
                f"""Transform the following structured podcast section into a spontaneous dialogue between a host named {host_persona} and a guest named {guest_persona}, naturally including conversational fillers, pauses with possible duration in brackets [pause: 2s], and interaction tags.

                Section: {section}
                Content: {section_content}

                Include conversational tags like <laughter>, <overlap>, <interruption>, <hesitations> naturally."""
            )
        response = deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert at writing spontaneous and realistic podcast dialogues."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content


    def generate_dialogue_zeroshot(self, section, section_content, host_persona, guest_persona):
        prompt = (
            f"""Transform the following structured podcast section into a spontaneous dialogue between a host named {host_persona} and a guest named {guest_persona}, naturally including conversational fillers, pauses with possible duration in brackets [pause: 2s], and interaction tags.
            
            Section: {section}
            Content: {section_content}
            
            Include conversational tags like <laughter>, <overlap>, <interruption>, <hesitations> naturally and also include speaker identification like Host:<utt>, Guest: <utt>.
            """
        )
        response = deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert at writing spontaneous and realistic podcast dialogues."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content

In [ ]:
dialogue_generator=SpontaneousDialogueGenerator(deployment,"fanar")
enhancer=DialogueEnhancer(deployment,"fanar")

In [9]:
outline = json.loads(outline_text)

In [10]:
print(outline)

{'Intro1': {'description': 'Brief overview of the podcast and general welcome from the host.', 'script': ["Welcome to the podcast! I'm Alex, and I’m thrilled to bring you engaging discussions on relevant topics in education."]}, 'Intro2': {'description': "Introduction of today's topic and brief bio of the guest speaker.", 'script': ['Today, we’re discussing a groundbreaking topic: AI in Education—how technology is reshaping classrooms and paving the way for new possibilities.', 'Joining us is Jamie, a technology enthusiast and the CEO of an innovative startup specializing in AI solutions for learning.']}, 'Points': {'talking_points': {'Point Title 1': {'discussion': 'How AI is personalizing learning experiences for students.', 'questions': ['How does AI adapt to different student learning styles?', 'Could you give examples of tools or platforms excelling in personalized learning?'], 'response_hint': 'Jamie could highlight adaptive learning systems like intelligent tutoring platforms an

In [11]:
dialogue_styles = {
    "formal": {
        "host": "Good afternoon, Dr. Patel. Thank you for joining us to discuss the implications of artificial intelligence in public policy.",
        "guest": "Good afternoon, and thank you for the invitation. It’s a pleasure to contribute to this critical conversation."
    },
    "casual": {
        "host": "Hey, Alex! Great to have you on the show. We’ve got a lot to unpack about social media trends today.",
        "guest": "Hey! Super excited to be here. Let’s dive in!"
    },
    "humorous": {
        "host": "Welcome back, folks! Today we’re talking AI—yup, the thing that might steal your job, but can’t steal your coffee. Yet.",
        "guest": "Unless it learns how to queue at Starbucks, I think we’re safe—for now!"
    },
    "educational": {
        "host": "Can you break down what ‘machine learning’ actually means, for listeners who might be new to the concept?",
        "guest": "Of course! Simply put, machine learning is a way for computers to learn patterns from data, kind of like how humans learn from experience."
    },
    "inspirational": {
        "host": "You’ve overcome major obstacles to lead groundbreaking research. What kept you going through the toughest times?",
        "guest": "Honestly, the belief that what we’re building could change lives. That purpose kept me focused."
    }
}

In [12]:
style_examples_dialogues=dialogue_styles['inspirational']
podcast_dialogues = []
# Introductions
for key in ["Intro1", "Intro2"]:
    enhanced_intro = enhancer.enhance_section(
        key,
        " ".join(outline[key]["script"]),
        criteria="Engaging, energetic, welcoming, and enthusiastic"
    )
    podcast_dialogues.append({"section": key, "dialogue": enhanced_intro})

# Main Discussion
tp_key = "Points"
talking_points = outline[tp_key]["talking_points"]
for tp_title, tp_content in talking_points.items():
    section_content = f"{tp_title}: {tp_content['discussion']} " + \
                      " ".join([f"Q: {q}" for q in tp_content["questions"]])
    if style_examples_dialogues:  
        spontaneous_dialogue = dialogue_generator.generate_dialogue(
            tp_title, section_content, host_persona, guest_persona, style_examples_dialogues
        )
    else:                
        spontaneous_dialogue = dialogue_generator.generate_dialogue(
            tp_title, section_content, host_persona, guest_persona
        )
    podcast_dialogues.append({"section": tp_title, "dialogue": spontaneous_dialogue})
# Enhance conclusion
conclusion_key = "Con"
enhanced_conclusion = enhancer.enhance_section(
    conclusion_key,
    " ".join(outline[conclusion_key]["script"]),
    criteria="Powerful, thoughtful, reflective, and thought-provoking"
)
podcast_dialogues.append({"section": conclusion_key, "dialogue": enhanced_conclusion})


In [13]:
podcast_dialogues

[{'section': 'Intro1',
  'dialogue': '**Podcast Intro Script:**\n\n🎙️ *[Upbeat music fades in]*  \n\n"Hey there, friends and listeners! 🎉 Welcome to the show! I’m Alex, and I couldn’t be more excited to have you here! This is *the* go-to place for buzzing conversations, eye-opening insights, and fresh takes on all things education. 🚀  \n\nWhether you’re an educator, a lifelong learner, or just curious about the world around you, we’ve got something special just for YOU. So get comfy, turn up the volume, and let’s dive right into today’s thought-provoking journey together. Thanks for tuning in — trust me, you don’t want to miss a second of this!"  \n\n🎙️ *[Music swells and fades out]*'},
 {'section': 'Intro2',
  'dialogue': '**Intro2 - Modified Version**\n\n🎙️ *[Upbeat and engaging music fades out]*  \n\n**Host:** Hey there, listeners! Welcome back to another exciting episode of [Podcast Name]! Buckle up, because today, we’re diving headfirst into a topic that’s transforming the world a

In [ ]:
def modify_dialogues_by_persona(dialogues, persona_criteria):
    modified_dialogues = []
    all_dialogues=[]
    for dialogue in dialogues:
        all_dialogues.extend(dialogue['dialogue'])
        
    modified_dialogues = enhancer.enhance_section(
        "Overall Podcast",
        all_dialogues,
        criteria=persona_criteria
    )
    return modified_dialogues

def save_podcast_to_json(podcast, filename="podcast2.json"):
    with open(filename, 'w') as f:
        json.dump(podcast, f, indent=4)


In [20]:
modified_podcast = modify_dialogues_by_persona(podcast_dialogues, "Conversational, friendly, and engaging")

In [25]:
save_podcast_to_json(modified_podcast)

In [16]:
modified_podcast

"Sure! Here's your engaging, conversational reworking of the provided podcast script:\n\n---\n\n🎙️ [Upbeat music transitions in]\n\n**Host Alex**: Hey there, wonderful listeners! Welcome back to another exciting episode of [Podcast Name], the place where buzzing conversations meet fresh insights and cool takes on the fascinating world of education. 🚀 \n\nI'm Alex, your podcast host and curious mind navigating the ever-evolving intersection of technology and learning. Honestly, I'm downright thrilled to have you here! So, take a deep breath, grab your favorite beverage, find a comfy spot, and let's dive right into today’s thought-provoking journey together! 🎉\n\nToday’s topic? Drumroll, please... *AI in Education*! Yes, we're going straight into the AI revolution happening in classrooms. Imagine this: a future where technology doesn’t just support, but actively empowers educators and students. Sounds a little sci-fi, doesn't it? But guess what—it's happening. Right. Now.\n\nJoining me t